In [1]:
import numpy as np
import pandas as pd
import os, sys, json
import twint
import nest_asyncio
nest_asyncio.apply()

In [2]:
def tweets2df(tweets, country):
    def tweet2json(tweet):
        return {
            'id': tweet.id,
            'timestamp': tweet.timestamp,
            'username': tweet.username,
            'tweet': tweet.tweet,
            'replies_count': tweet.replies_count,
            'retweets_count': tweet.retweets_count,
            'likes_count': tweet.likes_count,
            'url': tweet.link,
            'country': country
        }
    
    jsons = [tweet2json(tweet) for tweet in tweets]
    df = pd.DataFrame(jsons)
    return df

In [3]:
countries = {
    'US': ['cnn', 'nytimes', 'huffpost', 'FoxNews'],
    'Brazil': ['g1', 'folha', 'Estadao', 'JornalOGlobo', 'JornaldoBrasil'],
    'Italy': ['repubblica', 'Corriere', 'Libero_official', 'virgilio_it'],
    'UK': ['BBCNews', 'guardian', 'MailOnline', 'Telegraph'],
    'China': ['ChinaDaily', 'PDChina', 'shanghaidaily', 'chinaorgcn']
}

In [4]:
fields = ['id', 'timestamp', 'username', 'tweet', 'replies_count', 'retweets_count', 'likes_count', 'url']
formatted = '"' + "\t".join(['{' + field + '}' for field in fields]) + '"'

In [6]:
datapath = '../data/twitter_data'

for country in countries.keys():
    newspapers = countries[country]
    for newspaper in newspapers:
        dfname = f'{country}_{newspaper}.tsv'
        if dfname in os.listdir(datapath):
            print(f'{newspaper} of {country} was skipped.')
            continue
            
        print(f'Running {newspaper} of {country}')
        tw = twint.Config()
        tw.Username = newspaper
        tw.Since = '2020-01-22'
        tw.Store_object = True
        tw.Stats = True
        tw.Hide_output = True

        twint.run.Search(tw)
        
        df = tweets2df(twint.output.tweets_list, country)
        df.to_csv(f'../data/twitter_data/{dfname}', sep='\t')
    
print(result)

cnn of US was skipped.
nytimes of US was skipped.
huffpost of US was skipped.
FoxNews of US was skipped.
g1 of Brazil was skipped.
folha of Brazil was skipped.
Estadao of Brazil was skipped.
JornalOGlobo of Brazil was skipped.
JornaldoBrasil of Brazil was skipped.
repubblica of Italy was skipped.
Corriere of Italy was skipped.
Libero_official of Italy was skipped.
virgilio_it of Italy was skipped.
BBCNews of UK was skipped.
Running guardian of UK
Running MailOnline of UK
Running Telegraph of UK
Running ChinaDaily of China
Running PDChina of China
Running shanghaidaily of China
Running chinaorgcn of China


NameError: name 'result' is not defined